In [ ]:
#| default_exp cli

In [ ]:
#| export
from fastcore.utils import *
from fastcore.script import call_parse, bool_arg
from subprocess import check_output, run

import json

In [ ]:
from fastcore.test import *

In [ ]:
#| export
@call_parse
def railway_link():
    "Link the current directory to the current project's Railway service"
    j = json.loads(check_output("railway status --json".split()))
    prj = j['id']
    idxpath = 'edges', 0, 'node', 'id'
    env = nested_idx(j, 'environments', *idxpath)
    svc = nested_idx(j, 'services', *idxpath)

    cmd = f"railway link -e {env} -p {prj} -s {svc}"
    res = check_output(cmd.split())

In [ ]:
#| export
def _run(a, **kw):
    print('#', ' '.join(a))
    run(a)

In [ ]:
#| export
@call_parse
def railway_deploy(
    name:str, # The project name to deploy
    mount:bool_arg=True # Create a mounted volume at /app/data?
):
    "Deploy the current directory to Railway"
    cp = run("railway status --json".split(), capture_output=True)
    if not cp.returncode: return print("This project is already deployed. Run `railway open`.")
    reqs = Path('requirements.txt')
    if not reqs.exists(): reqs.write_text('python-fasthtml')
    _run(f"railway init -n {name}".split())
    _run(f"railway up -c".split())
    _run(f"railway domain".split())
    railway_link.__wrapped__()
    _run(f"railway volume add -m /app/data".split())    

## Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()